In [10]:
import requests
from bs4 import BeautifulSoup
import pymysql

db = pymysql.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    passwd = "753951DnDn!!",
    db = "playground",
    charset = "utf8"
)

cursor = db.cursor()

create_table_sql = """
CREATE TABLE IF NOT EXISTS books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(200),
    price_gbp DECIMAL(10,2),
    stock VARCHAR(200),
    rating VARCHAR(30),
    category VARCHAR(100),
    product_url VARCHAR(200)
);
"""
cursor.execute(create_table_sql)
db.commit()

base_url ="https://books.toscrape.com/catalogue/category/books/"
categories = {
    "science": "science_22",
    "mystery": "mystery_3",
    "travel": "travel_2"
}


for category_name, category_url in categories.items():
    for page in range(1,3) :
        if page == 1 :
            url = f"{base_url}{category_url}/index.html"
        else :
            url = f"{base_url}{category_url}/page-{page}.html"
            
        response = requests.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code != 200:
            print(f"페이지없음 : {url}")
            continue
            
        soup = BeautifulSoup(response.text, "html.parser")
        books = soup.select("article.product_pod")
        
        for book in books :
            title = book.h3.a["title"]
            price_gbp = book.select_one(".price_color").text.replace("£", "")
            stock = book.select_one(".availability").text.strip()
            rating = book.p.get("class")[1]
            category = category_name
            relative_url = book.h3.a["href"]
            product_url = "https://books.toscrape.com/catalogue/" + relative_url.lstrip("../")

            insert_sql = """
            INSERT INTO books (title, price_gbp, stock, rating, category, product_url)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_sql, (title, float(price_gbp), stock, rating, category, product_url))

        
db.commit()
cursor.close()
db.close()
print("✅ 크롤링 및 저장 완료")

OperationalError: (1054, "Unknown column 'category' in 'field list'")